In [ ]:
import cv2
import numpy as np

# Load YOLOv3 weights and configuration
weights_path = "yolov3.weights"
config_path = "yolov3.cfg"
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# Load COCO class labels
classes_path = "coco.names"
with open(classes_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set up the network
output_layers = net.getUnconnectedOutLayersNames()

# Load video
video_path = "video.mp4"
video = cv2.VideoCapture(video_path)

# Define output video properties
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
output_video = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

while True:
    ret, frame = video.read()

    if not ret:
        break

    height, width, channels = frame.shape

    # Perform blob conversion and forward pass through the network
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialization
    class_ids = []
    confidences = []
    boxes = []

    # Extract bounding box information and confidence scores
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maximum suppression to remove redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels on the frame
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # BGR color format

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the frame with bounding boxes to the output video
    output_video.write(frame)

# Release resources
video.release()
output_video.release()

print("Processing complete!")
